# <center> **Titanic**

# **Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings

# **Display Parameters**

## **Display Features**

In [ ]:
%matplotlib inline

pd.options.display.max_rows = 300000
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 500

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

## **Colors**

In [ ]:
color_1 = "bisque"
color_2 = "crimson"
color_3 = "orangered"
color_4 = "lightcoral"
color_5 = "royalblue"
color_6 = "pink"
color_7 = "indianred"
color_8 = "slategrey"
color_9 = "salmon"
color_10 = "beige"
color_11 = "coral"
color_13 = "grey"
color_14 = "tan"
color_15 = "wheat"
color_16 = "tomato"

## **Figure Parameters**

In [ ]:
size = 20

params = {
    "font.family": "Times New Roman",
    "font.size": size,
    "axes.labelsize": size,
    "xtick.labelsize": size * 0.75,
    "ytick.labelsize": size * 0.75,
    "figure.titlesize": size * 1.5,
    "axes.titlesize": size * 1.5,
    "axes.titlepad": size,
    "axes.labelpad": size - 10,
    "lines.linewidth": 2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.spines.left": False,
    "axes.spines.bottom": False,
    "legend.fontsize": size,
    "figure.figsize": (10, 6),
}

# **Data Overview and Preprocessing**

In [ ]:
data = pd.read_csv(
    r"C:\Users\Dell\Documents\Stroke\Data\stroke.csv",
    index_col=0
)

random_state = 101
target = 'Stroke'

In [ ]:
data.head()

## **Number of Rows and Columns**

In [ ]:
data.shape

## **Dataset Features**

1. **Gender:** Gender of the patient (Male or Female).
2. **Age:** Age of the patient (float).
3. **Hypertension:** 0 for no hypertension diagnosis. 1 for hypertion diagnosis.
4. **Heart Disease:** 0 for no diagnosis for heart disease. 1 for heart disease diagnosis.
5. **Ever Married:** Yes: The patient has been married or is married. No: The patient has never been married.
6. **Work:** Type of work: Private, Self-Employed, Government, Never Worked, Children (The patient is a child)
7. **Residence:** Two values: Rural, Urban
8. **AVG Glucose:** Average glucose level of the patient.
9. **BMI:** Body Mass Index of the patient.
10. **Smoking:**: Does the patient smoke now or before. Values: Unknown, Formerly Smoked, Never Smokes, Smokes.

## **Missing Data**

In [ ]:
data.isnull().sum(axis=0)

## **Duplicate Data**

In [ ]:
data[data.duplicated(keep=False)].sum()

# **Descriptive Information**

### **Data Types**

In [ ]:
data.info()

### **Descriptive Information for Numerical Features**

In [ ]:
data.drop(columns=['Stroke']).describe(include="number").applymap("{:,.2f}".format)

### **Descriptive Information for Categorical Features**

In [ ]:
data.describe(include="object")

### **Outliers**

An outlier is an observation that is unlike the other observations.

I used the **Interquartile Range (IQR)** method to identify outliers. The IQR is calculated as the difference between the 75th and the 25th percentiles of the data and defines the box in a box and whisker plot. I chose to show the outliers numerically instead of graphically. I saw more value in this type of presentation.

In [ ]:
numeric_data = data.select_dtypes(include=['number'])

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)

IQR = Q3 - Q1

outliers = (numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))

outlier_counts = outliers.sum()
print (outlier_counts)


### **Section Summary**
> * There are over 5,000 records in this dataset.
> * Of the 10 features, 7 are categorical, and 3 are numerical.  
> * The target is the column 'Stroke', with values 0 or 1. 
> * There are 201 null values in the BMI column. The other 9 columns have no null values.
> * Using the IQR method to identify outliers, I identified outliers in every numerical feature, except Age.  

# **Functions**

Below are the functions that I utilized in this project.

## **Side-by-Side Bar Plots**

In [ ]:
def side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette):

    '''
    Creates a side-by-side bar plot comparing two datasets.
    '''

    plt.rcParams.update(params)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    sns.barplot(data=data_1, x=feature, y=y, ax=ax1, palette=palette)
    sns.barplot(data=data_2, x=feature, y=y, ax=ax2, palette=palette)

    ax1.set_xlabel(feature)
    ax1.set_ylabel(y)
    ax2.set_xlabel(feature)
    ax2.set_ylabel(y)

    total_count1 = data_1[y].sum()
    for container in ax1.containers:
        labels = [f'{(v.get_height() / total_count1 * 100):.1f}%' for v in container]
        ax1.bar_label(container, labels=labels, size=size)

    total_count2 = data_2[y].sum()
    for container in ax2.containers:
        labels = [f'{(v.get_height() / total_count2 * 100):.1f}%' for v in container]
        ax2.bar_label(container, labels=labels, size=size)

    ax1.set_title(title_1)
    ax2.set_title(title_2)

    sns.despine()

    plt.show()

## **Distributions**

In [ ]:
def create_distributions(data, feature, target, value):

    '''
    Creates a distribution of counts for a specific feature within a subset of data.
    '''
    
    df = data[data[target] == value]
    distribution= df.groupby(feature).size().reset_index()
    distribution.columns = [feature, 'Count']

    return distribution

## **Create Bins**

In [ ]:
def create_bins(df, feature, new_feature, bins, labels):

    '''
    Creates bins for continuous features.
    '''

    df[new_feature] = pd.cut(df[feature], bins=bins, labels=labels, right=False)

    return df

## **Count Bins**

In [ ]:
def count_bins(df, new_feature):

    '''
    Counts the number of observations in each bin.
    '''
    
    group_counts = df[new_feature].value_counts().sort_index()
    group_counts_df = group_counts.reset_index()
    group_counts_df.columns = [new_feature, 'Count']

    return group_counts_df


## **Heat Map**

In [ ]:
def create_heatmap(data, title):

    '''
    Creates a Seaborn heatmap.
    '''

    plt.rcParams.update(params)
    corr = data.corr()

    mask = np.triu(np.ones_like(corr, dtype=bool))

    f, ax = plt.subplots(figsize=(25, 25))

    cmap = sns.diverging_palette(230, 10, as_cmap=True)
    heatmap = sns.heatmap(
        corr,
        mask=mask,
        vmax=1,
        vmin=-1,
        center=0,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.5},
        annot=True,
        cmap=plt.cm.Reds,
    )

    heatmap.set_title(
        title,
        fontdict={"fontsize": size},
        pad=12,
    )
    plt.xlabel("")
    plt.ylabel("")

## **Mutual Information**

In [ ]:
def create_plot_mi_scores(features, mi_scores):
    
    '''
    Creates a plot of mutual information scores.
    '''

    plt.rcParams.update({'figure.autolayout': True}) 

    scores = pd.Series(mi_scores, name="MI Scores", index=features.columns)
    scores = scores.sort_values(ascending=False)

    plt.figure(figsize=(15, 6))
    scores.plot(kind="line", marker='o')

    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.title("Mutual Information Scores")
    plt.xlabel("Feature")
    plt.ylabel("MI Score")

    plt.xticks(ticks=range(len(scores)), labels=scores.index, rotation=45, ha='right')

    plt.tight_layout()

    plt.show()

## **Two-Sample T-Test**

In [ ]:
def two_sample_t_test(sample1, sample2, variance):
    
    """
    Determines if the means of two samples are significanlty different.
    """
    
    if variance is False:
        print("The variance of the samples are different.")
    else:
        print("The variance of the samples are the same.")

    result = stats.ttest_ind(sample1, sample2, equal_var=variance)

    p_value = result.pvalue
    
    p_value = "{:.20f}".format(p_value)
    print("The p-value is: ", p_value)

    
    if result.pvalue < 0.05:
        print("Null hypothesis is rejected.")
    else:
        print("Failed to reject the null hypothesis.")

# **Exploratory Data Analysis**

## **Gender**

In this section, I show the distribution of genders among patients who suffered a stroke and those who did not.

In [ ]:
feature = 'Gender'
target = 'Stroke'
data = data.copy()  

In [ ]:
wi_stroke_gender = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_gender = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = ['Male', 'Female']
palette = [color_1, color_2]
y = 'Count'

data_1 = wi_stroke_gender
data_2 = no_stroke_gender
title_1 = 'Gender Distribution of Stroke Patients'
title_2 = 'Gender Distribution of Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette);

### **Section Summary**
> * Gender distribution is almost the same in patients who suffered a stroke and those who did not. 
> * From this data we can suggest that the medical establishments should look at both genders equally when assessing this ailment.

## **Hypertension**

In [ ]:
feature = 'Hypertension'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_hypertension = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_hypertension = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = [0, 1]
palette = [color_3, color_4]
y = 'Count'

data_1 = wi_stroke_hypertension.copy() 
data_2 = no_stroke_hypertension.copy()
data_1 ['Hypertension'] = data_1['Hypertension'].replace({0: 'No', 1: 'Yes'})
data_2 ['Hypertension'] = data_2['Hypertension'].replace({0: 'No', 1: 'Yes'})
title_1 = 'Hypertension Among Stroke Patients'
title_2 = 'Hypertension Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette);

### **Section Summary**
> * The percentage of patients with hypertension who also suffered a stroke is more than twice that of patients who did not.  
> * Hypertension may be one of the indicators for a doctor to consider as a possible risk factor when it comes to identifying the likelihood of stroke. 

## **Heart Disease**

In [ ]:
feature = 'Heart Disease'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_heart = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_heart = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = [0, 1]
palette = [color_5, color_6]
y = 'Count'

data_1 = wi_stroke_heart
data_2 = no_stroke_heart 

data_1 ['Heart Disease'] = data_1['Heart Disease'].replace({0: 'No', 1: 'Yes'})
data_2 ['Heart Disease'] = data_2['Heart Disease'].replace({0: 'No', 1: 'Yes'})
title_1 = 'Heart Disease Among Stroke Patients'
title_2 = 'Heart Disease Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette)

### **Section Summary**
> * The percentage of patients with heart disease who also suffered a stroke is more than three times of this number for patients who did not.  
> * Looking at this data, a doctor may consider heart disease as a possible risk factor when it comes to identifying the likelihood of stroke. 

## **Marital Status**

In [ ]:
feature = 'Ever Married'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_marriage = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_marriage = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = ['No', 'Yes']
palette = [color_7, color_8]
y = 'Count'

data_1 = wi_stroke_marriage
data_2 = no_stroke_marriage
title_1 = 'Marital Status Among Stroke Patients'
title_2 = 'Marital Status Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette)

### **Section Summary**
> * It is an accepted idea that our psychological and social states can affect our biology and our susecptibility to illness.   
> * Why is there nearly 3 times as many non-married people among patients who did not suffer a stroke? 
> * It is hard to say, but it gives us some reasons to look at the marital status of patients and the quality of such a relationship when it comes to identifying risk factors for stroke.

## **Work Status**

In [ ]:
feature = 'Work'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_work = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_work = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = ['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked']
palette = [color_1, color_2, color_3, color_9, color_10]
y = 'Count'

data_1 = wi_stroke_work
data_2 = no_stroke_work
title_1 = 'Work Status Among Stroke Patients'
title_2 = 'Work Status Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette)

### **Section Summary**
> * The only significant difference here is that a big percentage of those who did not suffer a stroke are those too young to work. 
> * This will be significant later in the analysis. Perhaps we can eliminate the group below a certain age since they are not likely to have a stroke. 

## **Residence Type**

In [ ]:
feature = 'Residence'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_residence = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_residence = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = ['Urban', 'Rural']
palette = [color_9, color_10]
y = 'Count'

data_1 = wi_stroke_residence
data_2 = no_stroke_residence
title_1 = 'Residence Type Among Stroke Patients'
title_2 = 'Residence Type  Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette)

### **Section Summary**
> * It seems that the place of residence (Urban or Rural) is not a significant factor.

## **Smoking Habits**

In [ ]:
feature = 'Smoking'
target = 'Stroke'
data = data.copy()

In [ ]:
wi_stroke_smoking = create_distributions(data, feature, target, value = 1)

In [ ]:
no_stroke_smoking = create_distributions(data, feature, target, value = 0)

In [ ]:
labels = ['formerly smoked', 'never smoked', 'smokes', 'Unknown']   
palette = [color_8, color_9, color_10, color_11]
y = 'Count'
data_1 = wi_stroke_smoking
data_2 = no_stroke_smoking
title_1 = 'Smoking Stroke Patients'
title_2 = 'Smoking Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, feature, y, palette)

### **Section Summary**
> * These figures seem interesting. 
> * Although a higher percentage of those who formerly smoked are in the stroke category, the percentage of current smokers are about the same in both groups. 
> * I did not expect that. I was sure that smoking will be a significant risk factor. 


## **Glucose Bins**

In [ ]:
feature = 'AVG Glucose'
target = 'Stroke'
new_feature = 'Glucose Bins'

bins = [50, 100, 150, 200, 250, 300]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

In [ ]:
df = data.copy()
data = create_bins(df, feature, new_feature, bins, labels)
data[new_feature] = data[new_feature].astype('object')

In [ ]:
df = data[data[target] == 1].copy()
wi_stroke_glucose = count_bins(df, new_feature)

In [ ]:
df = data[data[target] == 0].copy()
no_stroke_glucose = count_bins(df, new_feature)

In [ ]:
palette = [color_9, color_10, color_11, color_13, color_14]
y = 'Count'

data_1 = wi_stroke_glucose
data_2 = no_stroke_glucose
title_1 = 'Glucose Levels Among Stroke Patients'
title_2 = 'Glucose Levels Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, new_feature, y, palette)

### **Section Summary**
> * It seems that average glucose could be another risk factor contributing to stroke. 
> * 101 - 150: In this range, the average glucose levels of stroke and non-stroke patients do not seem to be very different. In fact, there is a slightly higher perectage of non-stroke patients at this glucose level.
> * 151 - 200: More than twice the percentage of stroke patients have this average glucose level.
> * 201 - 250: The percentage of stroke patients with this range of average glucose is nearly 3 times than those who did not suffer a stroke.


# **BMI Bins**

In [ ]:
feature = 'BMI'
new_feature = 'BMI Bins'
bins = [10, 20, 30, 40, 50, 70]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

In [ ]:
df = data.copy()
data = create_bins(df, feature, new_feature, bins, labels)
data[new_feature] = data[new_feature].astype('object')

In [ ]:
df = data[data[target] == 1]
wi_stroke_BMI = count_bins(df, new_feature)

In [ ]:
df = data[data[target] == 0]
no_stroke_BMI = count_bins(df, new_feature)

In [ ]:
palette = [color_1, color_2, color_3, color_4, color_5,color_9, color_10, color_11, color_13, color_14]
y = 'Count'

data_1 = wi_stroke_BMI
data_2 = no_stroke_BMI
title_1 = 'BMI Levels Among Stroke Patients'
title_2 = 'BMI Levels Among Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, new_feature, y, palette)

### **Section Summary**
> * Except at the lowest BMI range of 11 - 20, stroke and non-stroke patients seem to be nearly identical in their BMI scores. 

## **Age Bins**

In [ ]:
feature = 'Age'
new_feature = 'Age Bins'
bins = [0, 30, 50, 70, 80, 90]
labels = ['0-30', '30-50', '50-70', '70-80', '80-90']

In [ ]:
df = data.copy()
data = create_bins(df, feature, new_feature, bins, labels)
data[new_feature] = data[new_feature].astype('object')

In [ ]:
df = data[data[target] == 1]
wi_stroke_age = count_bins(df, new_feature)

In [ ]:
df = data[data[target] == 0]
no_stroke_age = count_bins(df, new_feature)

In [ ]:
palette = [color_1, color_2, color_3, color_4, color_5,color_9, color_10, color_11, color_13, color_14]
y = 'Count'

data_1 = wi_stroke_age
data_2 = no_stroke_age
title_1 = 'Age of Stroke Patients'
title_2 = 'Age of Non-Stroke Patients'

side_by_side_barplot(data_1, data_2, title_1, title_2, labels, new_feature, y, palette)

### **Section Summary**
> * It is obvious that a higher percentage of younger people are in the non-stroke group. 
> * In the 60 - 70 range, the percentage of stroke sufferers is more than three times than that patients who did not suffer a stroke. 
> * This clearly shows the role of age in this affliction. 

# **Hypothesis Tests**

## **Hypothesis Test: Smoking Habits and Stroke** 

### **Two-Proportion Z-Test**

The Two-Proportion Z-Test is used to determine whether there is a significant difference between the proportions of two independent groups. The two groups must be independent of each other. he test assumes that the sample size in each group is large enough to approximate the a normal distribution.

**Null**: There is no significant difference between the proporation of smokers who suffered a stroke and non-smokers.<BR>
**Alternative**: A higher proportion of smokers suffered a stroke compared to non-smokers. 

In [ ]:
feature = 'Smoking'
target = 'Stroke'

In [ ]:
smokers = data.loc[data[feature] == 'smokes']
count_smokers = smokers.shape[0]
count_smoker_stroke = (smokers[target] == 1).sum()
prop_smoker_stroke = count_smoker_stroke / count_smokers
print(
    f"Proportion of smokers who have suffered a stroke {prop_smoker_stroke:.3f}"
)

In [ ]:
nonsmokers = data.loc[data[feature] == 'never smoked']
count_nonsmokers = nonsmokers.shape[0]
count_nonsmoker_stroke = (nonsmokers[target] == 1).sum()
prop_nonsmoker_stroke = count_nonsmoker_stroke / count_nonsmokers
print(
    f"Proportion of nonsmokers who have suffered a stroke {prop_nonsmoker_stroke:.3f}"
)

In [ ]:
numerator = np.array([count_smoker_stroke, count_nonsmoker_stroke])
denominator = np.array([count_smokers, count_nonsmokers])

stat, pval = proportions_ztest(numerator, denominator, alternative="two-sided")

print(f"The p-value is: {pval:.2f}")

if pval< 0.05:
    print("Null hypothesis is rejected.")
else:
    print("Failed to reject the null hypothesis.")

### **Section Summary**
> * This high p-value suggests that there is no significant evidence to reject the null hypothesis. In other words, any observed effect or difference is likely due to random chance rather than a true effect.
> * The p-value of 0.54 is above the significance level threshold of 0.05 in the Standards section of this project.
> * Based on this p-values, the null hypothesis is not rejected. 
> * This fits with our observations earlier when looking at percentage of smokers and non-smokers among patients who suffered a stroke and those who did not. 

## **Hypothesis Test: Mean Glucose Level and Stroke**

### **Two-Sample T-Test**

The two-sample t-test, is used to determine if there is a significant difference between the means of two independent groups. The groups being compared must be independent, the data in both groups must be normally distributed and the variances of the two groups must be equal.

**Null**: There is no significant difference between the mean AVG Glucose of patients who suffered a stroke and patients who did not.<BR>
**Alternative**: The mean AVG Glucose of patients who suffered a stroke is significantly different than the mean AVG Glucose of patients who did not. 

In [ ]:
feature = 'AVG Glucose'
target = 'Stroke'

In [ ]:
no_stroke_glucose = data[data[target] == 0][feature]
wi_stroke_glucose = data[data[target] == 1][feature]

### **Numpy Variance Test**

In [ ]:
var_non_stroke_glucose = np.var(no_stroke_glucose)
var_stroke_glucose = np.var(wi_stroke_glucose)

if var_non_stroke_glucose == var_stroke_glucose:
    variance = True
else:
    variance = False

### **Two-Sample T-Test**

In [ ]:
pvalue = two_sample_t_test(no_stroke_glucose,wi_stroke_glucose, variance)

### **Section Summary**
> * This low p-value suggests that there may be significant evidence to reject the null hypothesis. In other words, any observed effect or difference is not likely due to random chance alone and may a true effect.
> * This suggests that average glucose level could a significant risk factor for stroke and it goes with our earlier observation of average glucose levels in patients who suffered a stroke and those who did not. 

# **Feature Engineering**

In this section, I create new features from existing ones in hopes of improving model performance.

In [ ]:
data_engineered = data.copy()

### **Missing Indicator**

In [ ]:
ami = AddMissingIndicator()
ami.fit(data_engineered)
data_engineered = ami.transform(data_engineered)
data_engineered.rename(columns={'BMI_na': 'BMI_NAN'}, inplace=True)
data_engineered.rename(columns={'BMI Bins_na': 'BMI Bins_NAN'}, inplace=True)

### **Random Sample Imputer**

In [ ]:
rsi = RandomSampleImputer()
rsi.fit(data_engineered)
data_engineered = rsi.transform(data_engineered)

### **Sum Glucose / Age**

In [ ]:
mf = MathFeatures(variables = ["AVG Glucose",'Age'], func = "sum")
mf.fit(data_engineered)
data_engineered= mf.transform(data_engineered)

### **Product Glucose / Age**

In [ ]:
mf = MathFeatures(variables = ["AVG Glucose",'Age'], func = "prod")
mf.fit(data_engineered)
data_engineered= mf.transform(data_engineered)

### **Mean Glucose / Age**

In [ ]:
mf = MathFeatures(variables = ["AVG Glucose",'Age'], func = "mean")
mf.fit(data_engineered)
data_engineered= mf.transform(data_engineered)

### **Hypertenstion OR Heart Disease**

In [ ]:
data_engineered['Hypertension or Heart_Disease'] = data_engineered['Hypertension'] | data_engineered['Heart Disease']

### **Hypertenstion AND Heart Disease**

In [ ]:
data_engineered['Hypertension and Heart_Disease'] = data_engineered['Hypertension'] & data_engineered['Heart Disease']

### **Label Encoder**

In [ ]:
label_encoder = LabelEncoder()
obj = data_engineered.dtypes == "object"

for col in list(obj[obj].index):
    data_engineered[col] = label_encoder.fit_transform(data_engineered[col])

### **Correlation between each Feature and the Target Feature**

In [ ]:
corr_matrix = data_engineered.corr(numeric_only=True)
corr_matrix['Stroke'].sort_values(ascending=False)

In [ ]:
title = "Correlation of Features"
create_heatmap(data_engineered, title)

### **Drop Collinear Features**

In [ ]:
dcf = DropCorrelatedFeatures(threshold=0.7)
data_engineered = dcf.fit_transform(data_engineered)

In [ ]:
corr_matrix = data_engineered.corr(numeric_only=True)
corr_matrix['Stroke'].sort_values(ascending=False)

In [ ]:
title = "Correlation of Features"
create_heatmap(data_engineered, title)

### **Mutual Information Plot**

This plot shows the predicitve power of each feature.

In [ ]:
features = data_engineered.drop('Stroke', axis=1)
target = data_engineered['Stroke']

mi_scores = mutual_info_classif(features, target, random_state=random_state)
create_plot_mi_scores(features, mi_scores)

### **Section Summary**
> * In this section, I dealth with missing values in the BMI and BMI related columns.
> * I created 5 additional features based on the existing features.
> * Using Feature Engine library, I dropped collinear features.
> * The mutual information column clearly identifies age as the major predictor of stroke.

# **Machine Learning**

In [ ]:
smote = SMOTE(random_state=random_state)

## **Eliminate White Space in Column Names**

In [ ]:
data_engineered.columns = [col.replace(' ', '_') for col in data_engineered.columns]

## **Train Test Split**

In [ ]:
X = data_engineered.drop('Stroke', axis=1)
y = data_engineered['Stroke']

X, y = shuffle(X, y, random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

X_train, y_train = smote.fit_resample(X_train, y_train)

## **K-Fold Cross Validation**

In [ ]:
columns_to_scale = ['Age', 'AVG_Glucose', 'BMI']
columns_to_impute = ['BMI']
X[columns_to_impute] = X[columns_to_impute].fillna(X[columns_to_impute].median())


preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', RobustScaler(), columns_to_scale)
    ],
    remainder='passthrough' 
)

svc_model = SVC(class_weight='balanced')
svc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc_model)
])


lg_model = LogisticRegression(class_weight='balanced', max_iter=5000)
lg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lg', lg_model)
])


xgb_model = XGBClassifier()
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', xgb_model)
])


lgbm_model = LGBMClassifier(class_weight='balanced', verbose=0)
lgbm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbm', lgbm_model)
])


rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state)
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', rf_model)
])


gb_model = GradientBoostingClassifier(n_estimators=100, random_state=random_state)
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gb', gb_model)
])


pipelines = {
    "SVC": svc_pipeline,
    "Logistic Regression": lg_pipeline,
    "XGBoost": xgb_pipeline,
    "LightGBM": lgbm_pipeline,
    "Random Forest": rf_pipeline,
    "Gradient Boosting": gb_pipeline
}


scorer = make_scorer(recall_score, pos_label=1)

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X, y, cv=10, scoring=scorer)
    print(f"{name}: Mean Recall:  {scores.mean():.2f}")

### **Section Summary**
> * In this section, I used cross valildation to test 6 different models on the engineered dataset.
> * Logistic Regression surpassed the other models with an accuracy score of 79%, also surpasoing my standard accuracy score of 75%.

## **Patients Over Age 50**

In [ ]:
data_overfifty = data_engineered[data_engineered['Age'] > 50]

### **Train Test Split**

In [ ]:
X = data_overfifty.drop('Stroke', axis=1)
y = data_overfifty['Stroke']

X, y = shuffle(X, y, random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

X_train, y_train = smote.fit_resample(X_train, y_train)

### **K-Fold Cross Validation**

In [ ]:
columns_to_scale = ['Age', 'AVG_Glucose', 'BMI']
columns_to_impute = ['BMI']
X[columns_to_impute] = X[columns_to_impute].fillna(X[columns_to_impute].median())


preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', RobustScaler(), columns_to_scale)
    ],
    remainder='passthrough' 
)

svc_model = SVC(class_weight='balanced')
svc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc_model)
])


lg_model = LogisticRegression(class_weight='balanced', max_iter=5000)
lg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lg', lg_model)
])


xgb_model = XGBClassifier()
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', xgb_model)
])


lgbm_model = LGBMClassifier(class_weight='balanced', verbose=0)
lgbm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbm', lgbm_model)
])


rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state)
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', rf_model)
])


gb_model = GradientBoostingClassifier(n_estimators=100, random_state=random_state)
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gb', gb_model)
])


pipelines = {
    "SVC": svc_pipeline,
    "Logistic Regression": lg_pipeline,
    "XGBoost": xgb_pipeline,
    "LightGBM": lgbm_pipeline,
    "Random Forest": rf_pipeline,
    "Gradient Boosting": gb_pipeline
}


scorer = make_scorer(recall_score, pos_label=1)

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X, y, cv=10, scoring=scorer)
    print(f"{name}: Mean Recall:  {scores.mean():.2f}")

### **Section Summary**
> * In this section, I used cross valildation to test 6 different models on the engineered dataset on a dataset limited to patient over 50, since the data showed that these patients are more likely to suffer a stroke.
> * None of the models surpassed the accuracy score of 75%, also surpasoing my standard accuracy score of 75%.

## **Age, Hypertension, Heart Disease, Average Glucose**

In [ ]:
columns_to_keep = ['Age', 'Hypertension', 'Heart_Disease','AVG_Glucose', 'Stroke']
data_mainfeatures = data_engineered[columns_to_keep]

### **Train Test Split**

In [ ]:
X = data_mainfeatures.drop('Stroke', axis=1)
y = data_mainfeatures['Stroke']

X, y = shuffle(X, y, random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

X_train, y_train = smote.fit_resample(X_train, y_train)

### **K-Fold Cross Validation**

In [ ]:
columns_to_scale = ['Age', 'AVG_Glucose']
# X[columns_to_impute] = X[columns_to_impute].fillna(X[columns_to_impute].median())


preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', RobustScaler(), columns_to_scale)
    ],
    remainder='passthrough' 
)

svc_model = SVC(class_weight='balanced')
svc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc_model)
])


lg_model = LogisticRegression(class_weight='balanced', max_iter=5000)
lg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lg', lg_model)
])


xgb_model = XGBClassifier()
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', xgb_model)
])


lgbm_model = LGBMClassifier(class_weight='balanced', verbose=0)
lgbm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbm', lgbm_model)
])


rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=random_state)
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', rf_model)
])


gb_model = GradientBoostingClassifier(n_estimators=100, random_state=random_state)
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gb', gb_model)
])


pipelines = {
    "SVC": svc_pipeline,
    "Logistic Regression": lg_pipeline,
    "XGBoost": xgb_pipeline,
    "LightGBM": lgbm_pipeline,
    "Random Forest": rf_pipeline,
    "Gradient Boosting": gb_pipeline
}


scorer = make_scorer(recall_score, pos_label=1)

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X, y, cv=10, scoring=scorer)
    print(f"{name}: Mean Recall:  {scores.mean():.2f}")

### **Section Summary**
> * In this section, I used cross valildation to test 6 different models on the engineered dataset on a limited data set of 4 features only, the 4 features that showed the most predictive power of all the other ones.
> * Both Logistic Regression and SVC surpassed the other models with an accuracy score of 79% and 82% respectively, also surpasoing my standard accuracy score of 75%.

## **Tuned SVC Hyperparameters**

### **SVC Pipeline Optimized**

In [ ]:
columns_to_scale = ['Age', 'AVG_Glucose']

preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', RobustScaler(), columns_to_scale)
    ],
    remainder='passthrough' 
)

svc_model_optimized = SVC(kernel='linear', gamma='auto', C=0.1, class_weight='balanced')
svc_pipeline_optimized = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc_model)
])


pipelines = {
    "SVC": svc_pipeline_optimized,
}

scorer = make_scorer(recall_score, pos_label=1)

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X, y, cv=10, scoring=scorer)
    print(f"{name}: Mean Recall:  {scores.mean():.2f}")

In [ ]:
svc_pipeline_optimized.fit(X, y)

recall_scorer = make_scorer(recall_score)
cv_results = cross_validate(svc_pipeline_optimized, X, y, cv=5, scoring={'recall': recall_scorer})

print("Recall scores for each fold:", cv_results['test_recall'])
print("Mean recall score:", cv_results['test_recall'].mean())

### **Classification Report**

In [ ]:
y_pred = svc_pipeline_optimized.predict(X_test)
recall = recall_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

### **Confusion Matrix**

In [ ]:
conf_matrix = ConfusionMatrixDisplay.from_estimator (svc_pipeline_optimized, X_test, y_test, cmap=plt.cm.Reds)

conf_matrix.ax_.set_xticks([0, 1])
conf_matrix.ax_.set_xticklabels(["No", "Yes"])
conf_matrix.ax_.set_yticks([0, 1])
conf_matrix.ax_.set_yticklabels(["No", "Yes"])

plt.title('Confusion Matrix')
plt.show()

### **Section Summary**
> * In this section, I tried to optimize my SVC model using hyperparameter tuning. The results were similar to those I received without tuning, 82% recall.
> * SVC showed the best performance of all the other models, so I used it as my final model for deployment.

# **Create a Pickle File for Streamlit Deployment**

In [ ]:
with open('svc_pipeline_optimized.pkl', 'wb') as file:
    pickle.dump(svc_pipeline_optimized, file)

### **Section Summary**
> * Using this final model, I created a Streamlit application for final deployment.

# **Conclusions**

Here is a summary of the conclusions that may be drawn from this report. 

>* **The Analysis of the Data:** I reviewed over 5,000 datapoint related to patients with stroke. <br> 
>* **The Goal of the Project:** The goal of this project was to find a model that could predict if a patient is likely to have a stroke with a recall score of 0.75 or higher.<br>
>* **Models:** I utilized numerous models and numerous ways to hyperparameter tuning.  I chose two simple models, Logistic Regressin and Support Vector Machines. I chose two boosting classifiers, LGBM and XGB and two ensemble models: Random Forest Classifier and Gradient Boosting Classifier.<br>
>* **Encoding:** For categorical data, I tried both Label Encoding and One-Hot Encoding. I did not see significant differences. I chose Label Encoding since the resulting table was more readable.  <br>
>* **Imputing Missing Data:** For imputing missing data, I tried mean, median, zero and random imputers. I saw no signinficant difference between them in the predictions of my models. I chose Randmo Imputer, since it I thought with such little information about the participants, it makes no sense to make any judgments about their features.  <br>
>* **Feature Engineering and Hyperparameter Testing:** I tried feature engineering and hyperparameter testing with techniques such as Backward Elimination, SHAP and OPTUNA. Some, I included in this report and some I didn't for sake of brevity. None of the measures I utilized improved resutls significanlty.<br> 
>* **Support Vector Classification:** For a simple model and using only default hyperparameters, SVC was able to get better or similar results than any other model, including the more complex ones.<br>  
>* **Boosting Models:** Of the boosting models that I utilized, none of them performed better than SVC.
>* **Ensemble Models:** Of the ensemble models that I utilized, none of them performed better than SVC.
>* **Recommendation:** I am not able to make any medical recommendations based on this data. However, some obvious elements that I was sure would contribute to the risk of stroke such as smoking or BMI, turned out to be a very poor predictors. <br> 

# **Suggestions for Improvement**

This report has certain weaknesses. In this section, I outlined those weaknesses and indicated some avenues for improvement. 

>* **Domain Knowledge:** It is best if the data scientist has adequate domain knowledge on the topic of the analysis. I do not have any expertise in the medical field. There may be parts of the data that I have overlooked that may have been important and I may have given importance to parts that may have had little significance. <br>
>* **More Detailed Data:** The data provide only general information on patients. This information is not adequate to predict a disease as complex as stroke. Information such as family history, genetic markers, blood trace element markers and more are missing in this data. More detailed information could have helped make better predictions. <br>  
>* **Balance:** The data is heavely imbalanced. Of the more than 5,000 datapoints, only about 300 are related to stroke patients. This, in addition to inadequcy of the data as mentioned above adds to the complexity predictions.  <br>  
>* **Visualizations:** If I had more time, I would improve on the bar graphs to emphasize certain data by using specific colors.  <br>  
>* **Functions:** I modularized most of the code in this notebook but not all due to limitation of time.  <br>  
>* **Statistics:** Continue to improve my statistical knowledge to create better analyses.<br>
>* **Pandas:** Continue to learn to utilize more optimized Pandas techniques and algorithms.<br>
>* **Seaborn and Matplotlib:** Continue to improve my knowledge of Seaborn and Matplotlib for creating visualizations. <br>
>* **Python Code:** Continue to write better and more efficient Python code. <br>
>* **Clean Code:** Continue to adhere to the principles of writing clean code. <br>
>* **Readability and Efficiency:** Continue to improve my skills to find the delicate balance between readability and efficiency in coding.<br>
>* **Functions File:** For my next project, I will create a file with my functions, separate from the notebook file, to keep the notebook as a more reasonable length.<br>

# **Appendix**

## **Model Development Techniques**

>* **Batch Inference:** In batch inference, predictions are made on a large set of data at scheduled intervals.
>* **Real-Time Inference:** In online inference, predictions are made in real-time as requests arrive.
>* **Hybrid Interface:** Combines batch and online inference to leverage the benefits of both batch and real-time Interface.
>* **Shadow Mode Deployment:** A new model runs in parallel with the existing model but does not affect production outcomes. The results are logged for comparison.
>* **Canary Deployment:** A small subset of users is exposed to the new model, while the majority continues to use the old model. Based on performance, the new model is gradually rolled out to all users.
>* **A/B Testing:** Two or more models are deployed simultaneously to different user segments to evaluate which model performs better.
>* **Blue-Green Deployment:** Two identical environments (blue and green) are maintained. The new model is deployed to the blue environment while the green remains live. After validation, traffic is switched to the blue environment.
>* **Multi-Armed Bandit:** An extension of A/B testing that dynamically allocates traffic to the best-performing model based on ongoing results.

## **Boosting Models Used in this Report**

>* **XGBoost (Extreme Gradient Boosting):** XGBoost (Extreme Gradient Boosting) uses the gradient boosting framework, where weak learners are sequentially added to the model, each one correcting the errors of its predecessor. XGBoost includes regularization terms (L1 and L2) to prevent overfitting and improve model generalization. XGBoost can handle missing data internally.<br>
>* **LightGBM (Light Gradient Boosting Machine):** LightGBM (Light Gradient Boosting Machine) is a gradient boosting framework developed by Microsoft, ightGBM can directly handle categorical features without needing to pre-process them into numerical forms.